In [ ]:
import os
import sys
DIR = os.path.normpath(os.getcwd() + os.sep + os.pardir) + '/MLQD'
path_to_file = str(DIR) + '/' + 'evolution.py'
if os.path.isfile(path_to_file):
    MLQD_DIR = DIR
else:
    MLQD_DIR = os.getcwd() + '/MLQD'
if MLQD_DIR not in sys.path:
    sys.path.append(MLQD_DIR)

In [ ]:
import numpy as np
from evolution import quant_dyn
import matplotlib.pyplot as plt

Here we consider the excitation energy transfer in FMO complex with the following Hamiltonian [1]

\begin{equation} 
	\boldsymbol{\rm H} = \boldsymbol{\rm H}_{\rm s} + \boldsymbol{\rm H}_{\rm env} + \boldsymbol{\rm H}_{\rm s-env} + \boldsymbol{\rm H}_{\rm reorg} \, , 
\end{equation}
with all Hamiltonian terms given below [2]

\begin{alignat}{2}
	&\boldsymbol{\rm H}_{\rm s}= \sum_{i}^{n} | i\rangle \epsilon_i \langle i | + \sum_{i,j=1, i\neq j}^{n} \rangle i | J_{ij} \langle j | \, , \\
	&\boldsymbol{\rm H}_{\rm{env}}=\sum_{i=1}^{n} \sum_{k=1} \left(\frac{1}{2} \boldsymbol{\rm P}_{k, i}^{2}+\frac{1}{2} \omega_{k, i}^{2} \boldsymbol{\rm Q}_{k, i}^{2}\right)\, , \\
	& \boldsymbol{\rm H}_{\rm s-env} =-\sum_{i=1}^{n} \sum_{k=1} | i \rangle | c_{k,i} \boldsymbol{\rm Q}_{k, i} \langle i | \, , \\
	&\boldsymbol{\rm H}_{\rm reorg}=\sum_{i=1}^{n} | i \rangle \lambda_{i} \langle i |  \, ,
\end{alignat}

where $\boldsymbol{\rm H}_{\rm s}$, $\boldsymbol{\rm H}_{\rm env}$, $\boldsymbol{\rm H}_{\rm s-env}$ and $\boldsymbol{\rm H}_{\rm reorg}$ denote system (BChl molecules) Hamiltonian, Hamiltonian of protein-environment, system-environment interaction Hamiltonian and the reorganization term, respectively. [2]  In above equation, $n$ is the number of sites (BChl molecules), $\epsilon_i$ is the energy of the $i$th site and $J_{ij}$ is the inter-site coupling between sites $i$ and $j$. [2]  $\boldsymbol{\rm P}_{k, i}$, $\boldsymbol{\rm Q}_{k, i}$ and $\omega_{k,i}$ are respectively conjugate momentum, coordinate and frequency of the environment mode $k$ associated with site $i$.  [2] In $\boldsymbol{\rm H}_{\rm s-env}$, each site is connected to its own environment. [2]  The $c_{k,i}$ is the strength of coupling between site $i$ and mode $k$ of its environment. [2]  The reorganization term $\boldsymbol{\rm H}_{\rm reorg}$ can be seen as a counter term that emerges from the interaction of the sites with the environment. [2]  It is added to stop further renormalization of the site energy $\epsilon_i$ by the environment. [2] In the reorganization term $\boldsymbol{\rm H}_{\rm reorg}$, $\lambda_{i}$ is the reorganization energy corresponding to site $i$. [2]

Refs.

[1] The Journal of chemical physics 130 (23), 234111 (2009)

[2] Nature Communications, 13, 1930 (2022)

## Generation of data
We consider the Drude–Lorentz spectral density 
$$ 	J_{\text {env}}(\omega)=2 \lambda \frac{\omega \gamma}{\omega^{2}+\gamma^{2}},$$
where $\gamma$ and  $\lambda$ denote the characteristic frequency and the reorganization energy, respectively.

We define a parameter space $\mathcal{D}$ with $\lambda, \gamma$, $T$ as its dimensions. The range of the dimensions are defined as: $\lambda=\{10$, $40$, $70$, $\dots$, $520\}$cm$^{-1}$, $\gamma=\{25$, $50$, $75$, $\dots$, $500\}$cm$^{-1}$, $T=\{30$, $50$, $70$, $\dots$, $510\}$K. For each initial excitation case (on site-1,6 and 8), we generate 500 trajectories where the parameters were chosen based on farthest point sampling. We use the local thermalising Lindblad master equation (LTLME) to generate our data. You can also get HEOM trajectories from our QD3SET-1 data set.

More are here [Predicting the future of excitation energy transfer in light-harvesting complex with artificial intelligence-based quantum dynamics](https://doi.org/10.1038/s41467-022-29621-w "Named link title")

## AIQD approach
We prepare our training using parameters $\{m$, $\gamma$,  $\lambda$,  $T$,  $f(t)\}$ where $m$ is a label for the initial excitation site. Here we use 0.1, 0.6 and 0.8 as a label to differentiate between site-1, site-6 and site-8 cases. $f(t)$ is the logistic function to normalize the dimension of time, i.e., 

$$f(t) = a/(1 + b \exp(-(t + c)/d))$$ 

where $a, b, c$ and  $d$ are constants. Check out the Supplementary Figure 3 of our AIQD papar [Predicting the future of excitation energy transfer in light-harvesting complex with artificial intelligence-based quantum dynamics](https://doi.org/10.1038/s41467-022-29621-w "Named link title"). 

For each time-step, the AIQD approach predicts and was trained on the corresponding reduced density matrix in the following format $\mathcal{R}[\rho_{11}(t)], \mathcal{R}[\rho_{1N}(t)], \mathcal{I}[\rho_{1N}(t)] \dots, \mathcal{R}[\rho_{1N}(t)], \mathcal{I}[\rho_{1N}(t)], \mathcal{R}[\rho_{22}(t)], \dots, \mathcal{R}[\rho_{2N}(t)], \mathcal{I}[\rho_{2N}(t)], \mathcal{R}[\rho_{33}(t)], \dots, \mathcal{R}[\rho_{3N}(t)],\mathcal{I}[\rho_{3N}(t)],\dots, \dots,   \mathcal{R}[\rho_{NN}(t)]$ where $N$ is the dimension of the reduced density matrix and $\mathcal{R}$ and $\mathcal{I}$ represent the real and imaginary parts of the off-diagonal terms, respectively.  

### 2. Training with preparation of training data and optimization of the CNN model

We provide 20 trajectories in ```Jupyter_Notebooks/fmo_data``` for the sake of demonstration. Each trajectory is propagated with LTLME method upto ```t= 10 ps``` with time-step ```dt = 0.005```. 

In [ ]:
param={ 
        'n_states': 8,                  # int:  Number of states (SB) or sites (FMO), default 2 (SB) and 7 (FMO).
        'time': 10,                     # float: Propagation time in picoseconds (ps) for FMO complex and in (a.u.) for spin-boson model
        'time_step': 0.005,             # float: Time-step for time-propagation. Default values are 0.05 (spin-boson model) and 0.005ps for FMO complex.
        'QDmodel': 'createQDmodel',     # string: createQDmodel, the dafault option is useQDmodel
        'QDmodelType': 'AIQD',          # string: Type of model. The default option is OSTL
        'prepInput' : 'True',           # str: Prepare input files from the data (Default False)
        'XfileIn': 'x_data',            # str: (Optional, npy file) The prepared X file will be saved at the provided file name 
        'YfileIn': 'y_data',            # str: (Optional, npy file) The prepared Y file will be saved at the provided file name 
        'numLogf': 2,                  # int: Number of Logistic function for the normalization of time dimension. Default value is 1.0.    
        'LogCa' : 1.0,                  # float: Coefficient "a" in the logistic function, default values is 1.0 (you may not provide it)
        'LogCb' : 15.0,                 # float: Coefficient "b" in the logistic function, default values is 15.0 (you may not provide it)
        'LogCc' : -1.0,                 # float: Coefficient "a" in the logistic function, default values is -1.0 (you may not provide it)
        'LogCd' : 1.0,                  # float: Coefficient "d" in the logistic function, default values is 1.0 (you may not provide it)
        'gammaNorm': 500,               # float: Normalizer for Characteristic frequency. Default value is 500 in the case of FMO complex and 10 in the case of spin-boson model. The same values are also adopted in the provided trained models  
        'lambNorm': 520,                # float: Normalizer for System-bath coupling strength. Default value is 520 (FMO complex) and 1 (SB model). The same values are also adopted in the provided trained models 
        'tempNorm': 500,                # float: Normalizer for temperature. Default value is 510 (FMO complex) and 1 (SB model). The same values are also adopted in the provided trained models.
        'systemType': 'FMO',            # str: (Not optional) Need to define, wether your model is spin-boson (SB) or FMO complex (FMO) 
        'hyperParam': 'True',           # str: Default is False, we can pass True (optimize the hyperparameters) or False (don't optimize and run with the default structure)
        'patience': 10,                 # Int: Patience for early stopping in CNN training 
        'epochs': 100,                  # int: Number of epochs for training or optimization
        'max_evals': 100,               # int: Number of maximum evaluations in hyperopt optimization
        'dataPath': 'fmo_data',          # str: Data path
        'QDmodelOut': 'AIQD_FMO_model'   # str: (Optional), providing a name to save the model at
        }
    
quant_dyn(**param)

### 2. Training with preparation of training data but No optimization of the CNN model

In [ ]:
param={ 
        'n_states': 8,                  # int:  Number of states (SB) or sites (FMO), default 2 (SB) and 7 (FMO).
        'time': 10,                     # float: Propagation time in picoseconds (ps) for FMO complex and in (a.u.) for spin-boson model
        'time_step': 0.005,             # float: Time-step for time-propagation. Default values are 0.05 (spin-boson model) and 0.005ps for FMO complex.
        'QDmodel': 'createQDmodel',     # string: createQDmodel, the dafault option is useQDmodel
        'QDmodelType': 'AIQD',          # string: Type of model. The default option is OSTL
        'prepInput' : 'True',           # str: Prepare input files from the data (Default False)
        'XfileIn': 'x_data',            # str: (Optional, npy file) The prepared X file will be saved at the provided file name 
        'YfileIn': 'y_data',            # str: (Optional, npy file) The prepared Y file will be saved at the provided file name 
        'numLogf': 2,                  # int: Number of Logistic function for the normalization of time dimension. Default value is 1.0.    
        'LogCa' : 1.0,                  # float: Coefficient "a" in the logistic function, default values is 1.0 (you may not provide it)
        'LogCb' : 15.0,                 # float: Coefficient "b" in the logistic function, default values is 15.0 (you may not provide it)
        'LogCc' : -1.0,                 # float: Coefficient "a" in the logistic function, default values is -1.0 (you may not provide it)
        'LogCd' : 1.0,                  # float: Coefficient "d" in the logistic function, default values is 1.0 (you may not provide it)
        'gammaNorm': 500,               # float: Normalizer for Characteristic frequency. Default value is 500 in the case of FMO complex and 10 in the case of spin-boson model. The same values are also adopted in the provided trained models  
        'lambNorm': 520,                # float: Normalizer for System-bath coupling strength. Default value is 520 (FMO complex) and 1 (SB model). The same values are also adopted in the provided trained models 
        'tempNorm': 500,                # float: Normalizer for temperature. Default value is 510 (FMO complex) and 1 (SB model). The same values are also adopted in the provided trained models.
        'systemType': 'FMO',            # str: (Not optional) Need to define, wether your model is spin-boson (SB) or FMO complex (FMO) 
        'hyperParam': 'False',          # str: Default is False, we can pass True (optimize the hyperparameters) or False (don't optimize and run with the default structure)
        'patience': 10,                 # Int: Patience for early stopping in CNN training 
        'epochs': 100,                  # int: Number of epochs for training or optimization
        'dataPath': 'fmo_data',         # str: Data path
        'QDmodelOut': 'AIQD_FMO_model'  # str: (Optional), providing a name to save the model at
        }
quant_dyn(**param)

### 3. Training without preparation of training data and optimization of the CNN model. 

In this example, we will not prepare the training data and will directly pass the X and Y files as were prepared in demonstration 1 or 2 

In [ ]:
param={ 
        'n_states': 8,                  # int:  Number of states (SB) or sites (FMO), default 2 (SB) and 7 (FMO).
        'QDmodel': 'createQDmodel',     # string: createQDmodel, the dafault option is useQDmodel
        'QDmodelType': 'AIQD',          # string: Type of model. The default option is OSTL
        'systemType': 'FMO',            # str: (Not optional) Need to define, wether your model is spin-boson (SB) or FMO complex (FMO) 
        'XfileIn': 'x_data',            # str: (Not Optional, npy file) The X file 
        'YfileIn': 'y_data',            # str: (Not Optional, npy file) The Y file  
        'hyperParam': 'False',             # bool: Default is False, we can pass True (optimize the hyperparameters) or False (don't optimize and run with the default structure)
        'patience': 10,                 # Int: Patience for early stopping in CNN training
        'epochs': 100,                  # int: Number of epochs for training or optimization
        'QDmodelOut': 'AIQD_FMO_model'   # str: (Optional), providing a name to save the model at
}
quant_dyn(**param)

# Propagating dynamics with the trained AIQD model 

Here we will demonstrate how to propagate dynamics with the model we trained above "AIQD_FMO_model.hdf5". We will provide the simulation parameters and the AIQD will predict the corresponding dynamics

In [ ]:
param={ 
        'initState': 1,                  # int:  Initial state with Initial Excitation case (only required in FMO complex case, Default is '1')
        'n_states': 8,                   # Int:  Number of states (SB) or sites (FMO). Default is 2 (SB) and 7 (FMO).
        'time': 10,                       # float: Propagation time in picoseconds (ps)  for FMO complex and in (a.u.) for spin-boson model
        'time_step': 0.005,             # float: Time-step for time-propagation (you are not restricted to the time-step used in the training data, however better stick to that for good accuracy) Default values are 0.1 (KRR SB), 0.05 (AIQD and OSTL for spin-boson model) and 0.005ps for FMO complex
        'numLogf': 2,                  # int: Number of Logistic function for the normalization of time dimension. Default value is 1.0.   
        'LogCa' : 1.0,                  # float: Coefficient "a" in the logistic function, default values is 1.0 (you may not provide it)
        'LogCb' : 15.0,                 # float: Coefficient "b" in the logistic function, default values is 15.0 (you may not provide it)
        'LogCc' : -1.0,                 # float: Coefficient "a" in the logistic function, default values is -1.0 (you may not provide it)
        'LogCd' : 1.0,                  # float: Coefficient "d" in the logistic function, default values is 1.0 (you may not provide it)
        'gamma': 100,                   # float: Characteristic frequency (in cm^-1 for the provided trained FMO models, in (a.u.) for spin-boson model)
        'lamb': 10,                     # float: System-bath coupling strength  (in cm^-1 for the provided trained FMO models, in (a.u.) for spin-boson model)
        'temp': 300,                    # float: temperature in K  (in Kilven for the provided trained FMO models, in (a.u.) for spin-boson model)
        'gammaNorm': 500,               # float: Normalizer for Characteristic frequency. Default value is 500 in the case of FMO complex and 10 in the case of spin-boson model. The same values are also adopted in the provided trained models  
        'lambNorm': 520,                # float: Normalizer for System-bath coupling strength. Default value is 520 (FMO complex) and 1 (SB model). The same values are also adopted in the provided trained models 
        'tempNorm': 500,                # float: Normalizer for temperature. Default value is 510 (FMO complex) and 1 (SB model). The same values are also adopted in the provided trained models.
        'QDmodel': 'useQDmodel',        # st: In MLQD, the dafault option is useQDmodel tells the MLQD to propagate dynamics with an existing trained model
        'QDmodelType': 'AIQD',               # st: The type of model we wanna use, here AIQD. The default option is OSTL
        'systemType': 'FMO',                 # str: (Not optional)  Need to define, wether your model is spin-boson (SB) or FMO complex (FMO) 
        'QDmodelIn': 'AIQD_FMO_model.hdf5',  # str: (Not Optional for useQDmodel), provide the name of the trained ML model
        'QDtrajOut': 'Qd_trajectory'         # str: (Optional), File name where the trajectory should be saved
        }
quant_dyn(**param)

In [ ]:
ref_dyn = np.load('test_set/FMO/8_initial-1_gamma-100.0_lambda-220.0_temp-190.0.npy')
pred_dyn = np.load('Qd_trajectory.npy')
t1 = np.real(ref_dyn[:,0]) # fs
t2 = pred_dyn[:,0]
ref_site_1_pop = ref_dyn[:,1] # population of site-1
pred_site_1_pop = pred_dyn[:,1] # population of site-1

plt.rcParams['font.size'] = '20'
plt.figure(figsize=(10,8))
plt.plot(t2, pred_site_1_pop)
plt.plot(t1/1000, ref_site_1_pop, '-.')
plt.xlabel(r'$t \, (ps)$')
plt.ylabel(r'$\rho_{11}$')
plt.legend(["Predicted (AIQD)", 'Ref(LTLME)'])
plt.title('Time evolution of the exciton \n population in FMO complex')